The aim of this notebook is to take the raw data and transform into a clean dataset set for modelling.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
order_numbers = pd.read_excel('data/data_task.xlsx', sheet_name='order_numbers')
transaction_data = pd.read_excel('data/data_task.xlsx', sheet_name='transaction_data')
reported_data = pd.read_excel('data/data_task.xlsx', sheet_name='reported_data')

# Clean the order numbers df
    - order df by date
    - ensure the order numbers value increase (create boolean column) 

In [3]:
order_numbers.shape

(856, 2)

In [4]:
order_numbers.head()

,date,order_number
0,2018-01-07,33841906
1,2018-01-22,34008921
2,2018-01-25,34397468
3,2018-02-06,34434432
4,2018-02-08,34579365


In [5]:
order_numbers.dtypes

date            datetime64[ns]
order_number             int64
dtype: object

In [6]:
def validate_order(df, date_col, order_col):
    # Ensure DataFrame is sorted by date
    df = df.sort_values(by=date_col).reset_index(drop=True)

    # Create Boolean column to check if order number is higher than the previous row
    df["order_increasing"] = df[order_col].diff() > 0  

    # Set the first row to True (as there's no previous row to compare)
    df.loc[0, "order_increasing"] = True 

    return df

In [7]:
orders1 = validate_order(order_numbers, 'date', 'order_number')
orders1

,date,order_number,order_increasing
0,2018-01-07,33841906,True
1,2018-01-22,34008921,True
2,2018-01-25,34397468,True
3,2018-02-06,34434432,True
4,2018-02-08,34579365,True
...,...,...,...
851,2022-12-26,89053562,True
852,2022-12-27,89078365,True
853,2022-12-28,89109007,True
854,2022-12-29,89139373,True


In [8]:
orders1.value_counts('order_increasing')

order_increasing
True     638
False    218
Name: count, dtype: int64

In [9]:
# drop all rows where order_increasing == False

orders2 = orders1[orders1['order_increasing'] == True]
orders2.value_counts('order_increasing')

order_increasing
True    638
Name: count, dtype: int64

In [10]:
orders2.head()

,date,order_number,order_increasing
0,2018-01-07,33841906,True
1,2018-01-22,34008921,True
2,2018-01-25,34397468,True
3,2018-02-06,34434432,True
4,2018-02-08,34579365,True


In [11]:
# drop the feature 'order_increasing'

orders3 = orders2.drop(columns=['order_increasing'])
orders3.head()

,date,order_number
0,2018-01-07,33841906
1,2018-01-22,34008921
2,2018-01-25,34397468
3,2018-02-06,34434432
4,2018-02-08,34579365


In [12]:
# Resample to quarterly frequency and aggregate using sum
# Set 'date' as the index for resampling
orders3 = orders3.set_index("date")

orders_quarterly = orders3.resample("Q").sum().reset_index()

orders_quarterly['quarter'] = orders_quarterly['date'].dt.to_period('Q').astype(str)

orders_quarterly = orders_quarterly.drop(columns=['date'])

orders_quarterly.head()

/var/folders/nv/lzp9ffbj6cs24kjrpcj0q1ym0000gn/T/ipykernel_37301/3873680223.py:5: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  orders_quarterly = orders3.resample("Q").sum().reset_index()


,order_number,quarter
0,455381458,2018Q1
1,922108174,2018Q2
2,890192139,2018Q3
3,1515523453,2018Q4
4,1081879820,2019Q1


In [13]:
orders_quarterly.shape

(20, 2)

# Transaction data

In [14]:
transaction_data.shape

(1826, 4)

In [15]:
transaction_data.dtypes

date                         datetime64[ns]
total_spend_index                   float64
gross_orders_index                  float64
weekly_active_users_index           float64
dtype: object

In [16]:
transaction_data.head()

,date,total_spend_index,gross_orders_index,weekly_active_users_index
0,2018-01-01,0.052537,0.158983,1.563652
1,2018-01-02,0.839176,1.430843,1.628314
2,2018-01-03,0.182855,0.476948,1.633740
3,2018-01-04,2.364675,1.907790,1.657706
4,2018-01-05,0.687484,1.271860,1.679863


In [17]:
def date_sorting(df, date_col):

    # Ensure DataFrame is sorted by date
    df = df.sort_values(by=date_col).reset_index(drop=True)

    return df

In [18]:
transaction1 = date_sorting(transaction_data, 'date')
transaction1.head()

,date,total_spend_index,gross_orders_index,weekly_active_users_index
0,2018-01-01,0.052537,0.158983,1.563652
1,2018-01-02,0.839176,1.430843,1.628314
2,2018-01-03,0.182855,0.476948,1.633740
3,2018-01-04,2.364675,1.907790,1.657706
4,2018-01-05,0.687484,1.271860,1.679863


In [19]:

# Set 'date' as the index for resampling
transaction1 = transaction1.set_index("date")

# Resample to quarterly frequency and aggregate using sum
transaction_quarterly = transaction1.resample("Q").sum().reset_index()

transaction_quarterly['quarter'] = transaction_quarterly['date'].dt.to_period('Q').astype(str)

transaction_quarterly = transaction_quarterly.drop(columns=['date'])

transaction_quarterly.head()

/var/folders/nv/lzp9ffbj6cs24kjrpcj0q1ym0000gn/T/ipykernel_37301/2499572133.py:5: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  transaction_quarterly = transaction1.resample("Q").sum().reset_index()


,total_spend_index,gross_orders_index,weekly_active_users_index,quarter
0,91.829498,110.810811,199.114172,2018Q1
1,355.545853,338.473768,462.304148,2018Q2
2,833.816717,829.888712,1195.017386,2018Q3
3,1396.528405,1508.426073,1811.129600,2018Q4
4,1227.264316,1342.607313,2250.209135,2019Q1


In [20]:
transaction_quarterly.shape

(20, 4)

# join the order number data onto the transaction data, (left join to keep all the transaction data)

In [21]:
joined_df = pd.merge(transaction_quarterly, orders_quarterly, how = 'left', on = 'quarter')
joined_df.head()

,total_spend_index,gross_orders_index,weekly_active_users_index,quarter,order_number
0,91.829498,110.810811,199.114172,2018Q1,455381458
1,355.545853,338.473768,462.304148,2018Q2,922108174
2,833.816717,829.888712,1195.017386,2018Q3,890192139
3,1396.528405,1508.426073,1811.129600,2018Q4,1515523453
4,1227.264316,1342.607313,2250.209135,2019Q1,1081879820


# Feature Engineering

In [22]:
# create a spend per user column
joined_df["spend_per_user"] = (
    joined_df["total_spend_index"] / joined_df["weekly_active_users_index"]
)

In [23]:
# create some lag variables

# one period back
joined_df["prev_spend_per_user"] = joined_df["spend_per_user"].shift(1)
joined_df["prev_order_volume"] = joined_df["order_number"].shift(1)
joined_df["prev_weekly_active_orders_index"] = joined_df["weekly_active_users_index"].shift(1)
joined_df["prev_total_spend_index"] = joined_df["total_spend_index"].shift(1)


In [24]:
joined_df.head()

,total_spend_index,gross_orders_index,weekly_active_users_index,quarter,order_number,spend_per_user,prev_spend_per_user,prev_order_volume,prev_weekly_active_orders_index,prev_total_spend_index
0,91.829498,110.810811,199.114172,2018Q1,455381458,0.461190,NaN,NaN,NaN,NaN
1,355.545853,338.473768,462.304148,2018Q2,922108174,0.769073,0.461190,4.553815e+08,199.114172,91.829498
2,833.816717,829.888712,1195.017386,2018Q3,890192139,0.697744,0.769073,9.221082e+08,462.304148,355.545853
3,1396.528405,1508.426073,1811.129600,2018Q4,1515523453,0.771081,0.697744,8.901921e+08,1195.017386,833.816717
4,1227.264316,1342.607313,2250.209135,2019Q1,1081879820,0.545400,0.771081,1.515523e+09,1811.129600,1396.528405


# Join the reported data

In [25]:
reported_data.head()

,period,start_date,end_date,revenue_index
0,2018 Q1,2018-01-01,2018-02-28,100.000000
1,2018 Q2,2018-03-01,2018-06-30,253.521127
2,2018 Q3,2018-07-01,2018-08-31,139.788732
3,2018 Q4,2018-09-01,2018-12-31,306.161972
4,2019 Q1,2019-01-01,2019-02-28,117.693662


In [ ]:
# remove the whitespace in the period column to ensure accurate join

reported_data['period'] = reported_data['period'].str.replace(r'\s+', '', regex=True)
reported_data.head()

,period,start_date,end_date,revenue_index
0,2018Q1,2018-01-01,2018-02-28,100.000000
1,2018Q2,2018-03-01,2018-06-30,253.521127
2,2018Q3,2018-07-01,2018-08-31,139.788732
3,2018Q4,2018-09-01,2018-12-31,306.161972
4,2019Q1,2019-01-01,2019-02-28,117.693662


In [30]:
main_df = pd.merge(reported_data, joined_df, how = 'left', left_on = 'period', right_on= 'quarter')
main_df.head()

,period,start_date,end_date,revenue_index,total_spend_index,gross_orders_index,weekly_active_users_index,quarter,order_number,spend_per_user,prev_spend_per_user,prev_order_volume,prev_weekly_active_orders_index,prev_total_spend_index
0,2018Q1,2018-01-01,2018-02-28,100.000000,91.829498,110.810811,199.114172,2018Q1,455381458,0.461190,NaN,NaN,NaN,NaN
1,2018Q2,2018-03-01,2018-06-30,253.521127,355.545853,338.473768,462.304148,2018Q2,922108174,0.769073,0.461190,4.553815e+08,199.114172,91.829498
2,2018Q3,2018-07-01,2018-08-31,139.788732,833.816717,829.888712,1195.017386,2018Q3,890192139,0.697744,0.769073,9.221082e+08,462.304148,355.545853
3,2018Q4,2018-09-01,2018-12-31,306.161972,1396.528405,1508.426073,1811.129600,2018Q4,1515523453,0.771081,0.697744,8.901921e+08,1195.017386,833.816717
4,2019Q1,2019-01-01,2019-02-28,117.693662,1227.264316,1342.607313,2250.209135,2019Q1,1081879820,0.545400,0.771081,1.515523e+09,1811.129600,1396.528405


In [28]:
main_df.to_csv('data/modelling_data.csv')